In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to C:\Users\Siim Markus
[nltk_data]     Marvet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Quick hack to fix GPU memory allocation
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
qa = pd.read_json("data/for_team/divided_columns/lemmatize_google_qa.json.xz")
jokes = pd.read_json("data/for_team/divided_columns/lemmatize_jokes.json.xz")
articles = pd.read_json("data/for_team/divided_columns/lemmatize_news.json.xz")

In [5]:
stop_words = stopwords.words('english')
stop_words.extend(["...", "'s", "wo", "n't", "'m", "ca", "'ll", "'re", "'ve", "'d", "ha", "´´", "´", "´´´", ",", "!", "'", ":", ";", '"', "\\", "``"])
stop_words.extend(["wa", "''", "the", "he", "my", "it"])

data_for_df = []

jokes_lemmas = jokes['lemmatize']

for i, joke in enumerate(jokes_lemmas):
    joke_words = [word for word in joke if word not in stop_words and word.isalpha()]
    joke_text = " ".join(joke_words)
    data_for_df.append({'text': joke_text, 'joke': 1, 'category': 0})
    
qa_lemmas = qa['lemmatize']

for i, q in enumerate(qa_lemmas):
    qa_words = [word for word in q if word not in stop_words and word.isalpha()]
    qa_text = " ".join(qa_words)
    data_for_df.append({'text': qa_text, 'joke': 0, 'category': 1})
    
articles_lemmas = articles['lemmatize']

for i, article in enumerate(articles_lemmas):
    article_words = [word for word in article if word not in stop_words and word.isalpha()]
    article_text = " ".join(article_words)
    article_text = re.sub("reporting .*", "", article_text)
    data_for_df.append({'text': article_text, 'joke': 0, 'category': 2})

In [6]:
df = pd.DataFrame(data_for_df)
df.shape

(1380351, 3)

In [7]:
df = df.drop_duplicates('text')
df.shape

(1283149, 3)

In [8]:
df.head()

,text,joke,category
0,seafood diet see food fish eat,1,0
1,shoe store al bundy fat woman came shoe store ...,1,0
2,stalin said dark humor like food everyone get,1,0
3,really hate dementia remember,1,0
4,hey guy got nice joke ya u,1,0


### Preparing data for Neural Network

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X, X_test, y, y_test = train_test_split(df['text'], df['joke'], test_size=0.05, stratify=df['joke'])

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [13]:
num_words = 15000
maxlen = 500

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train)

tokenized_X_train = tokenizer.texts_to_sequences(X_train)
tokenized_X_val = tokenizer.texts_to_sequences(X_val)
tokenized_X_test = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(tokenized_X_train, maxlen=maxlen)
X_val_pad = pad_sequences(tokenized_X_val, maxlen=maxlen)
X_test_pad = pad_sequences(tokenized_X_test, maxlen=maxlen)

In [14]:
y_train_vec = to_categorical(y_train)
y_val_vec = to_categorical(y_val)
y_test_vec = to_categorical(y_test)

In [15]:
X_train_pad.shape

(1097091, 500)

In [16]:
X_test_pad.shape

(64158, 500)

In [17]:
X_val_pad.shape

(121900, 500)

### Neural Network training and testing

Model taken from the tensorflow guide: https://www.tensorflow.org/guide/keras/rnn. Plan was to add layers and tune it further but the results indicated this wasn't necessary and instead we put more work into preprocessing the data further and analysis.

In [22]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=num_words, input_length=maxlen, output_dim=64))
model.add(layers.LSTM(35))
model.add(layers.Dense(2, activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 64)           960000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 35)                14000     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 72        
Total params: 974,072
Trainable params: 974,072
Non-trainable params: 0
_________________________________________________________________


In [23]:
with tf.device('/gpu:0'):
    model.fit(X_train_pad, y_train_vec, validation_data=(X_val_pad, y_val_vec), epochs=3, batch_size=1024)

Epoch 1/3
1072/1072 [==============================] - 109s 102ms/step - loss: 0.0883 - accuracy: 0.9664 - val_loss: 0.0581 - val_accuracy: 0.9800
Epoch 2/3
1072/1072 [==============================] - 109s 102ms/step - loss: 0.0513 - accuracy: 0.9822 - val_loss: 0.0545 - val_accuracy: 0.9815
Epoch 3/3
1072/1072 [==============================] - 110s 102ms/step - loss: 0.0425 - accuracy: 0.9853 - val_loss: 0.0538 - val_accuracy: 0.9819


In [24]:
predictions_probs = model.predict(X_test_pad)
predictions = [np.argmax(prob) for prob in predictions_probs]

In [25]:
round(sum(predictions == y_test) / len(predictions) * 100, 3)

98.128

In [27]:
from tensorflow.keras.models import load_model
        
model.save("model.h5")

uusMudel = load_model("model.h5")
print(uusMudel.summary())

predictions_probs = uusMudel.predict(X_test_pad)
predictions = [np.argmax(prob) for prob in predictions_probs]
round(sum(predictions == y_test) / len(predictions) * 100, 3)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 64)           960000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 35)                14000     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 72        
Total params: 974,072
Trainable params: 974,072
Non-trainable params: 0
_________________________________________________________________
None


98.128